In [1]:
#Load modules necessary

In [2]:
conda install -c conda-forge pyyaml


Retrieving notices: done
Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.3.1
    latest version: 25.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [11]:
conda install spacy

Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.7.0
    latest version: 25.11.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [13]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 74.4 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [14]:
!pip install torch
!pip install transformers
!pip install unbabel-comet
!pip install evaluate



  Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (114 kB)
Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl (14.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.5
    Uninstalling numpy-2.3.5:
      Successfully uninstalled numpy-2.3.5


In [4]:
import pandas as pd
import numpy as np
import torch
import transformers
import spacy
from comet import download_model, load_from_checkpoint
import evaluate
from transformers import AutoTokenizer, AutoModelForSequenceClassification


/Users/Hawraa_1/opt/anaconda3/envs/py311/lib/python3.11/site-packages/torchmetrics/utilities/imports.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [125]:
# LOAD DATA
# ------------------------------------------------
results = pd.read_csv("data/korean_translation2.csv")
results.head()

,korean,actual translation,Google translation,Microsoft translation,DeepL translation
0,아직 3분 시작된 건 아니죠? 그렇죠?,"My three minutes hasn't started yet, has it?","The 3 minutes haven't started yet, right? Right?","It's not three minutes yet, right? Right?","It's not quite three minutes in yet, is it?"
1,"네, 맘대로 시작하실 수 없습니다.","No, you can't start the three minutes.","Yes, you can't start it however you want.","Yes, you can't start as you want.","Yes, you can't just start at will."
2,"3분 다시 설정해주세요, 이건 반칙입니다.","Reset the three minutes, that's just not fair.","Please reset the time to 3 minutes, this is a ...","Please set it again for 3 minutes, this is a f...","Please reset the 3 minutes, this is cheating."
3,"어머나, 여기 참 냉정하네요","Oh my God, it's harsh up here.","Oh my, it's so cold in here.","Oh my God, it's so cold here","My goodness, this place is cold."
4,정말이지 긴장되네요,I mean I'm nervous enough as it is.,I'm really nervous.,I'm really nervous,I'm really nervous


In [112]:
INPUT_CSV = "data/korean_translation2.csv"        # your input file
OUTPUT_CSV = "data/translations_scored.csv"  # file to save with scores

SOURCE_COL = 'korean'
REF_COL = 'actual translation'
MT_COLUMNS = [
    'Google translation',
    'Microsoft translation', 'DeepL translation'
]

In [113]:
BLEURT_MODEL_NAME = "Elron/bleurt-base-512" 

In [114]:
for col in [SOURCE_COL, REF_COL] + MT_COLUMNS:
    if col not in results.columns:
        raise ValueError(f"Column {col} missing. CSV has: {results.columns.tolist()}")

sources    = results[SOURCE_COL].astype(str).tolist()
references = results[REF_COL].astype(str).tolist()

print(f"Loaded {len(results)} rows.")

Loaded 2821 rows.


In [97]:
# SETUP BLEURT
# ------------------------------------------------
print("Loading BLEURT...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

bleurt_tokenizer = AutoTokenizer.from_pretrained(BLEURT_MODEL_NAME)
bleurt_model     = AutoModelForSequenceClassification.from_pretrained(BLEURT_MODEL_NAME)
bleurt_model.to(device).eval()

def bleurt_score_batch(refs, cands, batch=16):
    scores = []
    for i in range(0, len(refs), batch):
        with torch.no_grad():
            inputs = bleurt_tokenizer(
                refs[i:i+batch], 
                cands[i:i+batch],
                padding=True, truncation=True, return_tensors="pt"
            ).to(device)
            logits = bleurt_model(**inputs).logits.squeeze(-1)
            scores.extend(logits.cpu().tolist())
    return scores

Loading BLEURT...


In [115]:
print("Loading COMET...")
comet_path  = download_model("Unbabel/wmt22-comet-da")
comet_model = load_from_checkpoint(comet_path)

Loading COMET...


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.6.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
Encoder model frozen.
/Users/Hawraa_1/opt/anaconda3/envs/py311/lib/python3.11/site-packages/pytorch_lightning/core/saving.py:197: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [135]:
for mt_col in MT_COLUMNS:
    print(f"\n=== Scoring MT column: {mt_col} ===")
    
    mt_output = results[mt_col].astype(str).tolist()

    # --- BLEURT ---
    print("Scoring BLEURT...")
    results[f"{mt_col}_bleurt"] = bleurt_score_batch(references, mt_output)

    # --- COMET ---
    print("Scoring COMET...")
    comet_data = [{"src": s, "mt": m, "ref": r} for s, m, r in zip(sources, mt_output, references)]
    # gpus=0 forces CPU; set gpus=1 if you have a GPU and it's configured
    comet_out  = comet_model.predict(comet_data, batch_size=16, gpus=1)
    
    results[f"{mt_col}_comet"] = comet_out.scores
    print(f"System COMET score for {mt_col}: {comet_out.system_score:.4f}")


=== Scoring MT column: Google translation ===
Scoring BLEURT...


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores


Scoring COMET...


Predicting: 0it [00:00, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
TOKENIZERS_PARALLELISM=(true | false)
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Predicting DataLoader 0: 100%|██████████| 177/177 [17:36<00:00,  5.97s/it]


System COMET score for Google translation: 0.8143

=== Scoring MT column: Microsoft translation ===
Scoring BLEURT...


...

System COMET score for Microsoft translation: 0.8057

=== Scoring MT column: DeepL translation ===
Scoring BLEURT...


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores


Scoring COMET...


Predicting: 0it [00:00, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
TOKENIZERS_PARALLELISM=(true | false)
Predicting DataLoader 0: 100%|██████████| 177/177 [2:45:23<00:00, 56.06s/it]


System COMET score for DeepL translation: 0.8123


In [136]:
results.to_csv(OUTPUT_CSV, index=False)
print(f"\nSaved full scored dataset to: {OUTPUT_CSV}")

# Show preview
show_cols = [SOURCE_COL, REF_COL] + MT_COLUMNS + \
            [f"{c}_bleurt" for c in MT_COLUMNS] + \
            [f"{c}_comet"  for c in MT_COLUMNS]

print(results[show_cols].head())


Saved full scored dataset to: data/translations_scored.csv
                    korean                              actual translation  \
0    아직 3분 시작된 건 아니죠? 그렇죠?    My three minutes hasn't started yet, has it?   
1      네, 맘대로 시작하실 수 없습니다.          No, you can't start the three minutes.   
2  3분 다시 설정해주세요, 이건 반칙입니다.  Reset the three minutes, that's just not fair.   
3          어머나, 여기 참 냉정하네요                  Oh my God, it's harsh up here.   
4               정말이지 긴장되네요             I mean I'm nervous enough as it is.   

                                  Google translation  \
0   The 3 minutes haven't started yet, right? Right?   
1          Yes, you can't start it however you want.   
2  Please reset the time to 3 minutes, this is a ...   
3                       Oh my, it's so cold in here.   
4                                I'm really nervous.   

                               Microsoft translation  \
0          It's not three minutes yet, right? Right?   
1                  Yes

In [29]:
import pandas as pd

# 1. Load your data
df = pd.read_csv("data/translations_scored.csv")  # change to your filename

# 2. List the BLEURT columns you want to normalize
bleurt_cols = ["Google translation_bleurt", "Microsoft translation_bleurt", "DeepL translation_bleurt"]  # <-- change to your real column names

# 3. Min–max normalize each BLEURT column and create new columns
for col in bleurt_cols:
    col_min = df[col].min()
    col_max = df[col].max()
    df[col + "_minmax"] = (df[col] - col_min) / (col_max - col_min)

# 4. Save updated data
df.to_csv("data/translations_scored_normalized.csv", index=False)
pd.read_csv("data/translations_scored_normalized.csv")


,korean,actual translation,Google translation,Microsoft translation,DeepL translation,Google translation_bleurt,Google translation_comet,Microsoft translation_bleurt,Microsoft translation_comet,DeepL translation_bleurt,DeepL translation_comet,cosine_Google_translation,cosine_Microsoft_translation,cosine_DeepL_translation,Google translation_bleurt_minmax,Microsoft translation_bleurt_minmax,DeepL translation_bleurt_minmax
0,아직 3분 시작된 건 아니죠? 그렇죠?,"My three minutes hasn't started yet, has it?","The 3 minutes haven't started yet, right? Right?","It's not three minutes yet, right? Right?","It's not quite three minutes in yet, is it?",0.109180,0.856831,-0.223568,0.760674,0.215006,0.785601,0.912729,0.814388,0.827700,0.604399,0.509906,0.645778
1,"네, 맘대로 시작하실 수 없습니다.","No, you can't start the three minutes.","Yes, you can't start it however you want.","Yes, you can't start as you want.","Yes, you can't just start at will.",-0.179017,0.744170,-0.173276,0.773100,-0.462301,0.746914,0.637112,0.684821,0.549571,0.498889,0.527346,0.393687
2,"3분 다시 설정해주세요, 이건 반칙입니다.","Reset the three minutes, that's just not fair.","Please reset the time to 3 minutes, this is a ...","Please set it again for 3 minutes, this is a f...","Please reset the 3 minutes, this is cheating.",-0.579378,0.783946,-0.773436,0.746450,-0.475596,0.816355,0.660077,0.531548,0.631793,0.352314,0.319229,0.388739
3,"어머나, 여기 참 냉정하네요","Oh my God, it's harsh up here.","Oh my, it's so cold in here.","Oh my God, it's so cold here","My goodness, this place is cold.",-0.456702,0.824750,-0.527967,0.875555,-0.204798,0.801129,0.526661,0.523677,0.482740,0.397227,0.404350,0.489529
4,정말이지 긴장되네요,I mean I'm nervous enough as it is.,I'm really nervous.,I'm really nervous,I'm really nervous,0.443310,0.815813,0.396191,0.808104,0.396191,0.808104,0.849992,0.843571,0.843571,0.726725,0.724819,0.713214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2816,감사합니다.,Thank you.,thank you,I appreciate it.,Thank you.,0.987746,0.947537,0.770399,0.817672,1.166716,0.980769,0.970060,0.726700,1.000000,0.926046,0.854583,1.000000
2817,감사합니다.,Thank you.,thank you,I appreciate it.,Thank you.,0.987746,0.947537,0.770399,0.817672,1.166716,0.980769,0.970060,0.726700,1.000000,0.926046,0.854583,1.000000
2818,감사합니다.,Thank you.,thank you,I appreciate it.,Thank you.,0.987746,0.947537,0.770399,0.817672,1.166716,0.980769,0.970060,0.726700,1.000000,0.926046,0.854583,1.000000
2819,감사합니다.,Thank you.,thank you,I appreciate it.,Thank you.,0.987746,0.671433,0.770399,0.541739,1.166716,0.652318,0.970060,0.726700,1.000000,0.926046,0.854583,1.000000


In [32]:
import pandas as pd

accuracy_score_table = pd.read_csv("data/translations_scored_normalized.csv")

# Weights
W_COMET  = 0.7
W_BLEURT = 0.2
W_COSINE = 0.1  # adjust as needed

# --------------------------------------------------------
# Detect MT systems based on *_comet columns ONLY
# (these have the "nice" names with spaces)
# --------------------------------------------------------
mt_systems = [
    col.replace("_comet", "")
    for col in accuracy_score_table.columns
    if col.endswith("_comet")
]

print("Detected MT systems:", mt_systems)
print("\nAverage accuracy scores:\n")

for system in mt_systems:
    comet_col  = f"{system}_comet"
    bleurt_col = f"{system}_bleurt_minmax"

    # cosine can be named with spaces or underscores
    cosine_candidates = [
        f"cosine_{system}",                      # e.g. "cosine_Google translation"
        f"cosine_{system.replace(' ', '_')}",    # e.g. "cosine_Google_translation"
    ]
    cosine_col = next((c for c in cosine_candidates 
                       if c in accuracy_score_table.columns), None)

    missing = []
    if comet_col not in accuracy_score_table.columns:
        missing.append(comet_col)
    if W_BLEURT != 0 and bleurt_col not in accuracy_score_table.columns:
        missing.append(bleurt_col)
    if W_COSINE != 0 and cosine_col is None:
        missing.append("cosine column")

    if missing:
        print(f"Skipping {system}: missing {missing}")
        continue

    comet_mean  = accuracy_score_table[comet_col].mean()
    bleurt_mean = accuracy_score_table[bleurt_col].mean() if W_BLEURT != 0 else 0.0
    cosine_mean = accuracy_score_table[cosine_col].mean() if W_COSINE != 0 else 0.0

    avg_score = (
        W_COMET  * comet_mean  +
        W_BLEURT * bleurt_mean +
        W_COSINE * cosine_mean
    )

    print(f"{system}: {avg_score:.4f}")


Detected MT systems: ['Google translation', 'Microsoft translation', 'DeepL translation']

Average accuracy scores:

Google translation: 0.7992
Microsoft translation: 0.7882
DeepL translation: 0.7664


,korean,actual translation,Google translation,Microsoft translation,DeepL translation,Google translation_bleurt,Google translation_comet,Microsoft translation_bleurt,Microsoft translation_comet,DeepL translation_bleurt,DeepL translation_comet,cosine_Google_translation,cosine_Microsoft_translation,cosine_DeepL_translation
0,아직 3분 시작된 건 아니죠? 그렇죠?,"My three minutes hasn't started yet, has it?","The 3 minutes haven't started yet, right? Right?","It's not three minutes yet, right? Right?","It's not quite three minutes in yet, is it?",0.109180,0.856831,-0.223568,0.760674,0.215006,0.785601,0.912729,0.814388,0.827700
1,"네, 맘대로 시작하실 수 없습니다.","No, you can't start the three minutes.","Yes, you can't start it however you want.","Yes, you can't start as you want.","Yes, you can't just start at will.",-0.179017,0.744170,-0.173276,0.773100,-0.462301,0.746914,0.637112,0.684821,0.549571
2,"3분 다시 설정해주세요, 이건 반칙입니다.","Reset the three minutes, that's just not fair.","Please reset the time to 3 minutes, this is a ...","Please set it again for 3 minutes, this is a f...","Please reset the 3 minutes, this is cheating.",-0.579378,0.783946,-0.773436,0.746450,-0.475596,0.816355,0.660077,0.531548,0.631793
3,"어머나, 여기 참 냉정하네요","Oh my God, it's harsh up here.","Oh my, it's so cold in here.","Oh my God, it's so cold here","My goodness, this place is cold.",-0.456702,0.824750,-0.527967,0.875555,-0.204798,0.801129,0.526661,0.523677,0.482740
4,정말이지 긴장되네요,I mean I'm nervous enough as it is.,I'm really nervous.,I'm really nervous,I'm really nervous,0.443310,0.815813,0.396191,0.808104,0.396191,0.808104,0.849992,0.843571,0.843571


In [138]:
#Cosine Similarity scores 
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


df = pd.read_csv("data/translations_scored.csv")

required_cols = [SOURCE_COL, REF_COL] + MT_COLUMNS
for c in required_cols:
    if c not in df.columns:
        raise ValueError(f"Missing required column: {c}")

# =========================================================
#  Load Embedding Model
# =========================================================
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# =========================================================
#  Encode the REFERENCE translations
# =========================================================
print("Encoding reference translations...")
ref_emb = model.encode(df[REF_COL].tolist(), show_progress_bar=True)
ref_emb = np.array(ref_emb)

# =========================================================
#  Prepare Output Columns
# =========================================================
for mt_col in MT_COLUMNS:
    score_col = f"cosine_{mt_col.replace(' ', '_')}"
    df[score_col] = np.nan

# =========================================================
#  Compute Embeddings + Cosine Similarity for Each MT System
# =========================================================
for mt_col in MT_COLUMNS:

    print(f"\nProcessing: {mt_col}")

    # Compute embeddings for MT output
    mt_emb = model.encode(df[mt_col].tolist(), show_progress_bar=True)
    mt_emb = np.array(mt_emb)

    # Compute cosine similarity for each pair
    cosine_scores = [
        cosine_similarity(ref_emb[i].reshape(1, -1),
                          mt_emb[i].reshape(1, -1))[0, 0]
        for i in range(len(df))
    ]

    # Create output column name
    score_col = f"cosine_{mt_col.replace(' ', '_')}"
    df[score_col] = cosine_scores

# =========================================================
#  Save Output
# =========================================================
df.to_csv(OUTPUT_CSV, index=False)
print(f"\nCosine similarity scores saved to: {OUTPUT_CSV}")


Use pytorch device_name: mps
Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2


Encoding reference translations...


Batches:   0%|          | 0/89 [00:00<?, ?it/s]


Processing: Google translation


Batches:   0%|          | 0/89 [00:00<?, ?it/s]


Processing: Microsoft translation


Batches:   0%|          | 0/89 [00:00<?, ?it/s]


Processing: DeepL translation


Batches:   0%|          | 0/89 [00:00<?, ?it/s]


Cosine similarity scores saved to: data/translations_scored.csv


In [17]:
df1 = pd.read_csv("data/translations_scored.csv")
df1.head()

,korean,actual translation,Google translation,Microsoft translation,DeepL translation,Google translation_bleurt,Google translation_comet,Microsoft translation_bleurt,Microsoft translation_comet,DeepL translation_bleurt,DeepL translation_comet,cosine_Google_translation,cosine_Microsoft_translation,cosine_DeepL_translation
0,아직 3분 시작된 건 아니죠? 그렇죠?,"My three minutes hasn't started yet, has it?","The 3 minutes haven't started yet, right? Right?","It's not three minutes yet, right? Right?","It's not quite three minutes in yet, is it?",0.109180,0.856831,-0.223568,0.760674,0.215006,0.785601,0.912729,0.814388,0.827700
1,"네, 맘대로 시작하실 수 없습니다.","No, you can't start the three minutes.","Yes, you can't start it however you want.","Yes, you can't start as you want.","Yes, you can't just start at will.",-0.179017,0.744170,-0.173276,0.773100,-0.462301,0.746914,0.637112,0.684821,0.549571
2,"3분 다시 설정해주세요, 이건 반칙입니다.","Reset the three minutes, that's just not fair.","Please reset the time to 3 minutes, this is a ...","Please set it again for 3 minutes, this is a f...","Please reset the 3 minutes, this is cheating.",-0.579378,0.783946,-0.773436,0.746450,-0.475596,0.816355,0.660077,0.531548,0.631793
3,"어머나, 여기 참 냉정하네요","Oh my God, it's harsh up here.","Oh my, it's so cold in here.","Oh my God, it's so cold here","My goodness, this place is cold.",-0.456702,0.824750,-0.527967,0.875555,-0.204798,0.801129,0.526661,0.523677,0.482740
4,정말이지 긴장되네요,I mean I'm nervous enough as it is.,I'm really nervous.,I'm really nervous,I'm really nervous,0.443310,0.815813,0.396191,0.808104,0.396191,0.808104,0.849992,0.843571,0.843571


In [12]:
min(df1['cosine_Google_translation']), max(df1['cosine_Google_translation']) 

(0.5266609, 0.9127289)

In [13]:
min(df1['Google translation_comet']), max(df1['Google translation_comet']) 

(0.7441701889038086, 0.8568310737609863)

In [14]:
min(df1['Google translation_bleurt']), max(df1['Google translation_bleurt']) 

(-0.5793778896331787, 0.4433097541332245)